In [ ]:
#include <vector>
#include <string>
#include <mutex>

class write_ahead_log{
public:
    void write(unsigned tag, const std::string& blob){
        std::unique_lock lock(mut);
        log.push_back(std::pair(tag,blob));
    }
    bool read(unsigned index, unsigned& tag, std::string& blob){
        std::unique_lock lock(mut);
        if(index<log.size()){tag=log[index].first; blob=log[index].second; return true;}
        return false;
    }
private:
    std::vector<std::pair<unsigned,std::string>> log;
    std::mutex mut;
};

In [ ]:
#include <vector>
#include <sstream>

class shared_state{
public:
    shared_state(write_ahead_log&_wal):wal(_wal),wal_index(0){}
public:
    class state_changer{
    friend class shared_state;
    public:
        state_changer(shared_state& s):state(s)
            {tag=state.changers.size();state.changers.push_back(this);}
        void change(){
            std::ostringstream out;
            on_save(out); state.wal.write(tag,out.str());
            state.update();
        }
    protected:
        virtual void on_save(std::ostream&) = 0;
        virtual void on_load(std::istream&) = 0;
        virtual void on_change() = 0;
    private:
        unsigned tag;
        shared_state& state;
    };    
public:
    void update(){
        unsigned tag; std::string blob; std::istringstream in;
        for(;wal.read(wal_index,tag,blob);wal_index++){
            state_changer& changer=*changers[tag];
            in.str(blob); changer.on_load(in);
            changer.on_change();
        }
    }
private:
    write_ahead_log& wal;
    unsigned wal_index;
    std::vector<state_changer*> changers;
};


In [1]:
#include <set>
#include <time.h>

// proposed annotation
#define _SHARED_STATE_
#define _STATE_CHANGER_
#define _IN_
#define _OUT_
#define _CONST_

template <class T>
class _SHARED_STATE_ scanner{
public:
	scanner(bool master):is_master(master),ready_to_compute(false){}
public:
	void scan() {
        srand(time(0));
        unsigned index = 0;
        
        if(is_master) { 
            for(auto& element: array)share_element(index++,element);
            set_ready_to_compute();
        }
        while(!ready_to_compute) update_state();
        
        while(get_not_scanned(index,rand())){
            on_scan(array[index]); put_scanned(index,array[index]);
        }
	}
	std::vector<T> array;
protected:	
    virtual void on_scan(T&element) = 0;
private:
    void _STATE_CHANGER_ update_state() _CONST_ {}

    void _STATE_CHANGER_ set_ready_to_compute() {
        not_scanned.clear();
        for(int i=0;i<array.size();i++)not_scanned.insert(i);
        ready_to_compute = true;
    }
    void _STATE_CHANGER_ share_element(unsigned _IN_ index, _IN_ T&element){
        if(!is_master){
            if(array.size()<=index) array.resize(index+1);
            array[index] = element;
        }
    }
    bool _STATE_CHANGER_ get_not_scanned(unsigned& _OUT_ index, int _IN_ random) _CONST_ {
        if(not_scanned.size()==0) return false;
        int selected = random % not_scanned.size();
        auto it=not_scanned.begin(); for(int i=0; i!=selected; i++,it++){} index=*it;
        return true;
    }
    void _STATE_CHANGER_ put_scanned(unsigned _IN_ index, _IN_ T&element){
        if(not_scanned.find(index)!=not_scanned.end()){//if(not_scanned.contains(index))
            array[index] = element; not_scanned.erase(index);
        }
    }
private:
    std::set<unsigned> not_scanned;
    bool is_master;
    bool ready_to_compute;
};

In [1]:
#include <iostream>
#include <vector>
#include <sstream>

#include <threads.h>
#include <stdlib.h>

template <class T>
class scanner{
public:
	scanner(bool master):is_master(master){}                
public:
	void scan() {
        if(is_master)
		for(auto& element: array){
			std::stringstream stream;
            
			on_save(element,stream,false);
			on_load(element,stream,false);

			on_scan(element);
			
			stream.clear();
			on_save(element,stream,true);
			on_load(element,stream,true);
		}
        else thrd_exit(EXIT_SUCCESS);
	}
	std::vector<T> array;
protected:	
    virtual void on_scan(T&element) = 0;
	virtual void on_save(T&element, std::ostream&, bool scanned) = 0;
	virtual void on_load(T&element, std::istream&, bool scanned) = 0;
private:
    bool is_master;
};

In [2]:
struct sqware_type{
	int N;
	int NxN;
}; 

class calc_sqware_scanner : public  scanner<sqware_type>{
public:
	calc_sqware_scanner(bool master):scanner(master){}
private:
	void on_scan(sqware_type&e) override { e.NxN = e.N * e.N; }
	void on_save(sqware_type&e, std::ostream&out, bool scanned) override {
		if(scanned) out << e.NxN; else out << e.N;
	}
	void on_load(sqware_type&e, std::istream&in, bool scanned) override {
		if(scanned) in >> e.NxN; else in >> e.N;
	}
};

In [3]:
#include <thread>
#include <atomic>
{
    std::atomic_int PID = -1;
    int NUM_THREADS = 10;
    
    //write_ahead_log wal;
    std::vector<std::thread> threads(NUM_THREADS);
    
    for(auto& t:threads)t=std::thread([&]{ PID++;
    //////////////// inside a 'process' ////////////////
        calc_sqware_scanner scanner_object(PID==0);
        
        if(PID==0){
        	scanner_object.array.resize(10);
        	for(int i=0;i<10;i++) scanner_object.array[i].N = i;
        }
    	
        scanner_object.scan();

    	for(int i=0;i<10;i++)
    		std::cout << scanner_object.array[i].N	<< " * " << scanner_object.array[i].N << " = "
    			<< scanner_object.array[i].NxN << std::endl;
    ////////////////////////////////////////////////////
    }); for(auto& t:threads) t.join();
}

0 * 0 = 0
1 * 1 = 1
2 * 2 = 4
3 * 3 = 9
4 * 4 = 16
5 * 5 = 25
6 * 6 = 36
7 * 7 = 49
8 * 8 = 64
9 * 9 = 81
